# Cheap Flight Chatbot

## Features
- Human-language interface to finding cheap flights.
- Uses tools calls to obtain data.

## How to run

You have to create a file called `.env` in the project's root.
Its content should be like:
```
OPENAI_API_KEY=sk-proj-...
```
(OpenAI key)


Create an environment (one time action):
```
conda env create -f environment.yaml 

```
Activate it:
```
conda activate llm-flight-bot
```

Run jupyter lab:
```
jupyter lab
```

## Disclaimers
This is PoC-type project. Its sole purpose is education.
Please adhere to the license agreements/terms of use when scraping websites. The one chosen here is just an example and one should be aware of terms of use before using it.

## Inspirations
LLM wrappers: 
https://github.com/ed-donner/llm_engineering/blob/main/week2/community-contributions/day1_class_definition-botChat.ipynb

Notebook structure: 
https://github.com/ed-donner/llm_engineering/blob/main/week2/community-contributions/day4-airlines-project-fullyCustomize.ipynb


In [ ]:
###############################################################################
# 1) Handle imports
###############################################################################

import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re
import json
import os
from abc import ABC, abstractmethod
from dotenv import load_dotenv
from openai import OpenAI
import ollama
from collections.abc import MutableSequence
from typing import TypedDict
import inspect
import gradio as gr


In [ ]:
###############################################################################
# 2) Create a Flight class - interact with cheap flights aggregator
###############################################################################
# Load JSON data once at module load
with open(os.path.join("static", "regions_cities.json")) as f:
    REGIONS_CITIES = json.load(f)

with open(os.path.join("static", "airports.json")) as f:
    AIRPORTS = json.load(f)

with open(os.path.join("static", "default_call_params.json")) as f:
    _default_params = json.load(f)
    BASE_URL = _default_params["URL"]
    DEFAULT_PARAMS = _default_params["PARAMS"]


class Flight:
    # class attributes
    BASE_URL: str = BASE_URL
    DEFAULT_PARAMS: dict = DEFAULT_PARAMS
    REGIONS_CITIES: dict = REGIONS_CITIES
    AIRPORTS: dict = AIRPORTS

    params: dict  # dictionary of parameters used to generate the search URL
    request_url: str  # URL used to fetch the flight search results
    response_text: str  # html returned from the flight provider's search results
    response_code: int  # status code of the response from the flight provider
    flights_parsed: list  # list of dictionaries containing parsed flight details

    def __init__(self, custom_params=None):
        self.params = {**self.DEFAULT_PARAMS, **(custom_params or {})}

    def __get_airports(self, code):
        if code in Flight.REGIONS_CITIES:
            return Flight.REGIONS_CITIES[code]["ports"].split("_")
        return [code]

    def __get_airport_name(self, code):
        return Flight.AIRPORTS.get(code, code)

    def __fetch_flights_html(self):
        if not hasattr(self, "request_url"):
            print(
                "Error: No URL generated. Please call create_search_flights_url first."
            )
            return None, None

        response = requests.get(self.request_url)
        self.response_code = response.status_code

        if response.status_code == 200:
            self.response_text = response.text
        else:
            print(f"Error: Received status code {response.status_code}")
            self.response_text = None
        return response.status_code, self.response_text

    def __parse_flight_details(self):
        def extract_flight_details(section):
            date_str = section.find_next(class_="date").text.strip()
            date = datetime.strptime(date_str, "%a %d/%m/%y")
            from_time_str = section.find_next(class_="from").find("strong").text.strip()
            from_time = datetime.strptime(from_time_str, "%H:%M").time()
            from_airport_code = (
                section.find_next(class_="from").find(class_="code").text.strip()
            )
            from_airport_name = self.__get_airport_name(from_airport_code)
            to_time_str = section.find_next(class_="to").text.split()[0].strip()
            to_time = datetime.strptime(to_time_str, "%H:%M").time()
            to_airport_code = (
                section.find_next(class_="to").find(class_="code").text.strip()
            )
            to_airport_name = self.__get_airport_name(to_airport_code)
            duration_str = section.find_next(class_="durcha").text.strip()
            duration_match = re.search(r"(\d+):(\d+)", duration_str)
            duration = timedelta(
                hours=int(duration_match.group(1)), minutes=int(duration_match.group(2))
            )
            changes = "no change" not in duration_str

            # Extract airline information from detail div
            detail_div = section.find_next("div", class_="detail")
            airline = None
            if detail_div:
                airline_span = detail_div.find("span", class_="airline")
                if airline_span:
                    airline = airline_span.text.strip()

            return {
                "date": date,
                "from_time": from_time,
                "from_airport_code": from_airport_code,
                "from_airport_name": from_airport_name,
                "to_time": to_time,
                "to_airport_code": to_airport_code,
                "to_airport_name": to_airport_name,
                "duration": duration,
                "changes": changes,
                "airline": airline,
            }

        if self.response_code != 200:
            print("Error: No response text to parse.")
            return None

        soup = BeautifulSoup(self.response_text, "html.parser")
        flights = []

        for result in soup.select(".list .result"):
            try:
                there_section = result.find("span", class_="caption tam")
                back_section = result.find("span", class_="caption sem")

                there_details = (
                    extract_flight_details(there_section.parent)
                    if there_section
                    else {}
                )
                back_details = (
                    extract_flight_details(back_section.parent) if back_section else {}
                )

                total_price_str = (
                    result.find(class_="totalPrice").find(class_="tp").text.strip()
                )
                total_price_value = float(total_price_str.split()[0].replace(",", "."))
                total_price_currency = total_price_str.split()[1]

                length_of_stay_str = result.find(class_="lengthOfStay").text.strip()
                length_of_stay_days = int(re.search(r"\d+", length_of_stay_str).group())

                # Extract Kiwi.com booking link
                reservation_link = None
                aff_links = result.find("div", class_="affLinksContainer")
                if aff_links:
                    kiwi_link = aff_links.find("a", {"name": "FlightExtBooking4"})
                    if kiwi_link:
                        reservation_link = kiwi_link.get("href")

                flights.append(
                    {
                        "there": there_details,
                        "back": back_details,
                        "total_price": {
                            "value": total_price_value,
                            "currency": total_price_currency,
                        },
                        "length_of_stay_days": length_of_stay_days,
                        "reservation_link": reservation_link,
                    }
                )
            except Exception as e:
                print(f"Error parsing flight: {e}")

        self.flights_parsed = flights
        return json.dumps(flights, indent=4, sort_keys=True, default=str)

    def get_flights(
        self,
        src_airport,
        dep_date,
        arr_date,
        min_days_stay=2,
        max_days_stay=10,
        max_changes=0,
        adults=1,
        src_additional_airports=[],
        dst_additional_airports=[],
        dst_airport="XXX",
        currency="PLN",
    ):
        try:
            dep_month = datetime.strptime(dep_date, "%Y-%m-%d").strftime("%Y%m")
            arr_month = datetime.strptime(arr_date, "%Y-%m-%d").strftime("%Y%m")
        except ValueError:
            print("Error: Invalid date format. Expected format is YYYY-MM-DD.")
            return None

        src_airports = self.__get_airports(src_airport)
        dst_airports = self.__get_airports(dst_airport)

        src_airport_name = (
            Flight.REGIONS_CITIES[src_airport]["name"]
            if src_airport in Flight.REGIONS_CITIES
            else self.__get_airport_name(src_airports[0])
        )
        dst_airport_name = (
            Flight.REGIONS_CITIES[dst_airport]["name"]
            if dst_airport in Flight.REGIONS_CITIES
            else self.__get_airport_name(dst_airports[0])
        )

        # Format additional airports with commas
        all_src_airports = src_airports + src_additional_airports
        additional_src_airports = (
            f" (+{','.join(all_src_airports[1:])})" if len(all_src_airports) > 1 else ""
        )
        all_dst_airports = dst_airports + dst_additional_airports
        additional_dst_airports = (
            f" (+{','.join(all_dst_airports[1:])})" if len(all_dst_airports) > 1 else ""
        )

        params = {
            **Flight.DEFAULT_PARAMS,
            "srcAirport": f"{src_airport_name} [{all_src_airports[0]}]{additional_src_airports}",
            "dstAirport": f"{dst_airport_name} [{all_dst_airports[0]}]{additional_dst_airports}",
            "srcTypedText": (
                src_airport.split("_")[0].lower()
                if src_airport in Flight.REGIONS_CITIES
                else ""
            ),
            "depmonth": dep_month,
            "depdate": dep_date,
            "arrmonth": arr_month,
            "arrdate": arr_date,
            "minDaysStay": min_days_stay,
            "maxDaysStay": max_days_stay,
            "adults": adults,
            "maxChng": max_changes,
            "currency": currency,
            "srcMC": src_airport if src_airport in Flight.REGIONS_CITIES else "",
        }

        # Add additional source airports
        for i, airport in enumerate(src_airports + src_additional_airports):
            params[f"srcap{i}"] = airport

        # Add additional destination airports
        for i, airport in enumerate(dst_airports + dst_additional_airports):
            params[f"dstap{i}"] = airport

        request_url = (
            requests.Request("GET", Flight.BASE_URL, params=params).prepare().url
        )
        self.request_url = request_url
        self.__fetch_flights_html()
        return self.__parse_flight_details()


In [ ]:
###############################################################################
# 3) Test the class (optional)
###############################################################################

# flight_api = Flight()
# flight_api.get_flights(
#         src_airport="PL_SOUTH",
#         src_additional_airports=[],
#         dst_airport="XXX",
#         dst_additional_airports=[],
#         dep_date="2025-03-01",
#         arr_date="2026-03-30",
#         min_days_stay=2,
#         max_days_stay=4,
#         adults=2,
#         max_changes=0,
#         currency="PLN"
# )


In [ ]:
###############################################################################
# 4) Wrappers for LLM interaction; inspiration: https://github.com/ed-donner/llm_engineering/blob/main/week2/community-contributions/day1_class_definition.ipynb
###############################################################################

class LLM_Wrapper(ABC):
    """
    The parent (abstract) class to specific LLM classes, normalising and providing common
    and simplified ways to call LLMs while adding some level of abstraction on
    specifics
    """

    MessageEntry = TypedDict("MessageEntry", {"role": str, "content": str})

    system_prompt: str  # The system prompt used for the LLM
    user_prompt: str  # The user prompt
    __api_key: str  # The (private) api key
    temperature: float = 0.5  # Default temperature
    __msg: MutableSequence[MessageEntry]  # Message builder

    def __init__(
        self, system_prompt: str, user_prompt: str, env_apikey_var: str = None
    ):
        self.system_prompt = system_prompt
        self.user_prompt = user_prompt
        if env_apikey_var:
            load_dotenv(override=True)
            self.__api_key = os.getenv(env_apikey_var)

    def set_system_prompt(self, prompt: str):
        self.system_prompt = prompt

    def set_user_prompt(self, prompt: str):
        self.user_prompt = prompt

    def set_temperature(self, temp: float):
        self.temperature = temp

    def get_key(self) -> str:
        return self.__api_key

    def message_set(self, message: MutableSequence[MessageEntry]):
        self.__msg = message

    def message_append(self, role: str, content: str):
        self.__msg.append({"role": role, "content": content})

    def message_get(self) -> MutableSequence[MessageEntry]:
        return self.__msg

    @abstractmethod
    def get_result(self, tools, format):
        pass

    @abstractmethod
    def get_result_generator(self, tools, format):
        pass

class GPT_Wrapper(LLM_Wrapper):

    MODEL: str = "gpt-4o-mini"
    llm: OpenAI

    def __init__(self, system_prompt: str = "", user_prompt: str = ""):
        super().__init__(system_prompt, user_prompt, "OPENAI_API_KEY")
        self.llm = OpenAI()
        super().message_set(
            [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.user_prompt},
            ]
        )

    def set_system_prompt(self, prompt: str):
        super().set_system_prompt(prompt)
        super().message_set(
            [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.user_prompt},
            ]
        )

    def set_user_prompt(self, prompt: str):
        super().set_user_prompt(prompt)
        super().message_set(
            [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.user_prompt},
            ]
        )

    def get_result(self, tools = None, format=None):
        response = self.llm.chat.completions.create(
            model=self.MODEL,
            messages=super().message_get(),
            temperature=self.temperature,
            response_format=format,  # eg {"type": "json_object"}
            tools=tools
        )
        if format and format["type"] == "json_object":
            result = json.loads(response.choices[0].message.content)
        else:
            result = response.choices[0].message.content
        return result

    def get_result_full(self, tools = None, format=None):
        response = self.llm.chat.completions.create(
            model=self.MODEL,
            messages=super().message_get(),
            temperature=self.temperature,
            response_format=format,  # eg {"type": "json_object"}
            tools=tools
        )
        if format and format["type"] == "json_object":
            result = json.loads(response)
        else:
            result = response
        return result

    def get_result_generator(self, tools = None, format=None):
        response = self.llm.chat.completions.create(
            model=self.MODEL,
            messages=super().message_get(),
            temperature=self.temperature,
            response_format=format,  # eg {"type": "json_object"}
            tools=tools
        )


class Ollama_Wrapper(LLM_Wrapper):

    MODEL: str = "llama3.2"

    def __init__(self, system_prompt: str = "", user_prompt: str = ""):
        super().__init__(system_prompt, user_prompt, None)
        self.llm = ollama
        super().message_set(
            [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.user_prompt},
            ]
        )

    def set_system_prompt(self, prompt: str):
        super().set_system_prompt(prompt)
        super().message_set(
            [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.user_prompt},
            ]
        )

    def set_user_prompt(self, prompt: str):
        super().set_user_prompt(prompt)
        super().message_set(
            [
                {"role": "system", "content": self.system_prompt},
                {"role": "user", "content": self.user_prompt},
            ]
        )

    def get_result(self, tools = None, format=None):
        """
        format is sent as an adittional parameter {"type", format}
        e.g. json_object
        """
        response = self.llm.chat(model=self.MODEL, messages=super().message_get(), tools = tools)
        result = response["message"]["content"]
        return result
    
    def get_result_full(self, tools = None, format=None):
        response = self.llm.chat(model=self.MODEL, messages=super().message_get(), tools = tools)
        result = response
        return result


In [ ]:
###############################################################################
# 5) Test LLM wrappers (optional)
###############################################################################

# test_llm_wrapper_system_msg1 = "You are an assistant who helps users find cheap flights, mainly in Europe."

# test_llm_wrapper_user_msg1 = """
# Please suggest a good destinations in Europe for late winter. 
# The climate there should be gentle and travellers should be able to spend time in 
# a variety of ways (e.g. in nature as well as sightseeing).
# """
# print(f"Test 1: {test_llm_wrapper_user_msg1}\n")
# test_gpt1=GPT_Wrapper(test_llm_wrapper_system_msg1, test_llm_wrapper_user_msg1)
# print(test_gpt1.get_result())

# print("-------------------\n")
# test_llm_wrapper_user_msg2 ="""
# Please suggest a good destinations in Europe for late winter. 
# I'm interested in lots of snow, winter sports activities, spending time in wilderness.
# Preferably the place should be cheap to stay in. 
# """
# print(f"Test 2: {test_llm_wrapper_user_msg2}\n")
# test_gpt1.set_user_prompt(test_llm_wrapper_user_msg2)
# print(test_gpt1.get_result())

# print("-------------------\n")
# test_llm_actor1_system_msg ="""
# You are a stereotypical philosopher.
# You doubt too much, always questioning even obvious things and exhibit sophistry.
# You over-complicate all things to appear smarter.
# Try not to use the same tactic all the time, but change approaches as the conversation goes.
# Once in a while use an illustrative example.
# Use up to 5 senteces.
# """
# test_llm_actor2_system_msg ="""
# You are a farmer - a man of few words, not interested in anything that isn't tangible and down-to-earth.
# You think that all philosophical ideas are useless and do not contribute too life.
# You respond in a succint manner and take life as it is and as you see it.
# But don't focus your answers only on farming, try to be a bit more versatile.
# Once in a while, you can ask a question to the philosopher. Sporadically, come up with a very clever response.
# Use up to 2 sentences, 3 only when adding an extra element.
# """
# test_llm_actor2_starter_message = """
# I would like to discuss life. My life, your life. Or is there any difference between our lives, perhaps they are all but one thing...?
# """
# print(f"Test 3: Chat. System msg of first actor: {test_llm_actor1_system_msg}\nSystem message of the second actor: {test_llm_actor2_system_msg}\n")
# test_gpt_philosopher = GPT_Wrapper()
# test_gpt_philosopher.set_system_prompt(test_llm_actor1_system_msg)
# test_gpt_farmer = GPT_Wrapper()
# test_gpt_farmer.set_system_prompt(test_llm_actor2_system_msg)
# test_gpt_farmer.set_user_prompt(test_llm_actor2_starter_message)

# # print philopsher starter
# print(f"Philosopher: {test_llm_actor2_starter_message}")

# # first farmer's response
# farmer_response = test_gpt_farmer.get_result()
# print(f"Farmer:\n{farmer_response}\n")
# test_gpt_farmer.message_append("assistant", farmer_response)

# for i in range(7):
#     test_gpt_philosopher.message_append("user", farmer_response)
#     philosopher_response = test_gpt_philosopher.get_result()
#     print(f"Philosopher:\n{philosopher_response}\n")
#     test_gpt_philosopher.message_append("assistant", philosopher_response)
#     # print(f"Philosopher messages after append: {test_gpt_philosopher.message_get()}")
    
#     test_gpt_farmer.message_append("user", philosopher_response)
#     farmer_response = test_gpt_farmer.get_result()
#     print(f"Farmer:\n{farmer_response}\n")
#     test_gpt_farmer.message_append("assistant", farmer_response)
#     # print(f"Farmer messages after append: {test_gpt_farmer.message_get()}")
    

In [ ]:
###############################################################################
# 6) Tools JSON Schemas
###############################################################################

flight_search_function = {
    "name": "search_flights",
    "description": "Search for flights based on user requirements. Makes reasonable assumptions for missing parameters.",
    "parameters": {
        "type": "object",
        "properties": {
            "src_airport": {
                "type": "string",
                "description": "Source airport or region code (e.g., 'KTW' for Katowice, 'WAW' for Warsaw)",
            },
            "dst_airport": {
                "type": "string",
                "description": "Destination airport or region code. Use 'XXX' for searching all destinations",
                "default": "XXX",
            },
            "dep_date": {
                "type": "string",
                "description": "Soonest departure date in YYYY-MM-DD format. Treat it like 'soonest departure date', does not mean exact date unless use says so.",
                "pattern": "^\\d{4}-\\d{2}-\\d{2}$",
            },
            "arr_date": {
                "type": "string",
                "description": "Latest return date in YYYY-MM-DD format. Treat it like 'latest arrival date', does not mean exact date unless use says so.",
                "pattern": "^\\d{4}-\\d{2}-\\d{2}$",
            },
            "min_days_stay": {
                "type": "integer",
                "description": "Minimum number of days to stay at destination",
                "minimum": 1,
                "default": 2,
            },
            "max_days_stay": {
                "type": "integer",
                "description": "Maximum number of days to stay at destination",
                "minimum": 1,
                "default": 10,
            },
            "max_changes": {
                "type": "integer",
                "description": "Maximum number of flight changes allowed",
                "minimum": 0,
                "maximum": 2,
                "default": 0,
            },
            "adults": {
                "type": "integer",
                "description": "Number of adult passengers",
                "minimum": 1,
                "maximum": 9,
                "default": 1,
            },
            "src_additional_airports": {
                "type": "array",
                "description": "Additional source airports to include in search, for example KRK for Krakow, WAW for Warsaw",
                "items": {"type": "string"},
                "default": [],
            },
            "dst_additional_airports": {
                "type": "array",
                "description": "Additional destination airports to include in search, for example MLA for Malta, BCN for Barcelona",
                "items": {"type": "string"},
                "default": [],
            },
            "currency": {
                "type": "string",
                "description": "Currency code for prices",
                "pattern": "^[A-Z]{3}$",
                "default": "PLN",
            },
        },
        "required": ["src_airport", "dep_date", "arr_date"],
        "additionalProperties": False,
    },
}

tools_chatbot = [
    {"type": "function", "function": flight_search_function}
]

In [ ]:
###############################################################################
# 7) Handle Tool Calls
###############################################################################
flight_api = Flight()

def handle_tool_call(message):
    """
    The LLM can request to call a function in 'tools'. We parse the JSON arguments
    and run the Python function. Then we return a 'tool' message with the result.
    """
    tool_call = message.tool_calls[0]
    fn_name   = tool_call.function.name
    args      = json.loads(tool_call.function.arguments)

    if fn_name == "search_flights":
        sig = inspect.signature(flight_api.get_flights) # obtain the signature of the function. This signature contains details about the parameters, including their names, default values, and annotations.
        call_args = {
            k: args.get(k, v.default)
            for k, v in sig.parameters.items()
            if k in args or v.default is not inspect.Parameter.empty
        }
        response_content = flight_api.get_flights(**call_args)

    else:
        response_content = {"error": f"Unknown tool: {fn_name}"}

    return {
        "role": "tool",
        "content": json.dumps(response_content),
        "tool_call_id": tool_call.id,
    }, args


In [ ]:
###############################################################################
# 8) System instructions
###############################################################################

# Load JSON data once at module load
with open(os.path.join( "static", "regions_cities.json")) as f:
    regions_cities = json.load(f)

with open(os.path.join( "static", "airports.json")) as f:
    airports = json.load(f)

airports_str = json.dumps(airports)
regions_cities_str = json.dumps(regions_cities)

system_message = f"""
    You are a helpful assistant for a user that wants to find cheap airline tickets.
    When the user wants to book a flight, follow these steps:
    1. Ask for the source airport. The user may give you more than 1 airport code.
    2. Ask for the destination city, use 'XXX' if users persists not to give it. The user may give you more than 1 airport code.
    3. Ask for the soonest departure date in YYYY-MM-DD format.
    4. Ask for the latest return date in YYYY-MM-DD format.
    5. Take note of information such as additional airports, number of adults, number of changes, currency, etc. They are optional, but may be provided.
    6. Call the function 'search_flights' with the necessary parameters. 
    7. Present results to the user, mention at least deparature and arrival dates, airports, airlines, price, duration of the flight, link to the reservation.
    8. Suggest a couple highlights, based on the price, attrictiveness of the destination, departure and arrival dates/hours (favor weekends and trips that minimize PTO one has to take).
    IMPORTANT:
    The user may use a language other than English, so be prepared to handle that. Respond in their language but prepare tools calls as you would for English.
    The list of available destinations/source airports are 'airports': {airports_str} as well as XXX destination, which means anywhere.
    'regions_cities': {regions_cities_str} contains 'multi airports' - the key is 'multi region' code and ports are airports that comprise is, separated by underscore. For example, if use says 'Paris', you can include all airports that have code in CDG, ORY, BVA, XCR, LBG. And you can get the name of the city from the 'airpots' variable you already now.
    Deduce airport based on general knowledge and 'airpots' and 'regions_cities' variables. For example, if a user says: any aiport close to London, then find any mention of London in any of the variables you have, see which codes they have and determine airports source names.
    If a user say something like "Close to Country/City" (e.g. "Close to Italy", "somewhere warm, maybe Greece?"), there won't be a code for that region, just search for XXX and use your knowledge to determine which countries and airports fit the description.
    Note that arrival date does not have to be as many days later than departure as long the trip is. If user wants 3 day trip for example, departure could be Jan 1 and arrival March 30. The engine will search in that period for 3 day trips automatically.
    If you don't know something, say so.
    Keep answers short and courteous.

    Example conversation:
    User: Hello
    Assistant: Hello! How can I help you today? Would you like to search for a cheap flight?
    User: yes, source airport Katowice or Krakow, anywhere
    Assistant: Great, I'll use Katowice as a main airport and Krakow as an additional one. I'll put XXX (anywhere) as a destination. When would you like to depart and return?
    User: I'd like to travel in March 2025.
    Assistant: Sure, then I'd suggest departing soonest on March 1st and returning latest on March 30th. How many days would you like to stay at the destination?
    User: weekend, 3-4 days
    Assistant: I'll look for trips that are 2-4 days long. Any other information you'd like to provide?
    User: 2 people travelling, thats all
    Assistant: Perfect! Let me search for flights for you now.
    [Assistant uses search_flights tool with parameters: {{"src_airport": "KTW", "src_additional_airports": "KRK", "dst_airport": "XXX", "dep_date": "2025-03-01", "arr_date": "2025-03-31", "adults": 2, "trip_days_min": 2, "trip_days_max": 4}}]
    Tool: [Returns flight data with 55 options]
    Assistant: I've found several great weekend getaway options for 2 people in March 2025:

    1. Barcelona (BCN): 432 PLN total - Depart KTW on Friday March 7, return Sunday March 9
    2. Rome (FCO): 354 PLN total - Depart KRK on Thursday March 20, return Sunday March 23
    3. London (STN): 288 PLN total - Depart KTW on Friday March 14, return Monday March 17
    
    The Barcelona option is not only the cheapest but also perfect for a weekend getaway without taking any time off work. Would you like more details on any of these options?
    User: Please search tell me if any options in Greece.
    Assistant: Sure, let me search for flights there.
    [Assistant uses search_flights tool with parameters: {{"src_airport": "KTW", "src_additional_airports": "KRK", "dst_airport": "GR", "dep_date": "2025-03-01", "arr_date": "2025-03-31", "adults": 2, "trip_days_min": 2, "trip_days_max": 4}}]
    Tool: [Returns flight data with 6 options]
    Assistant: I've found several options for 2 people in March 2025:

    1. Athens (ATH): 631 PLN total - Depart KRK on Friday March 7, return Sunday March 9
    2. Athens (ATH): 787 PLN total- Depart KRK on Thursday March 20, return Sunday March 23
    3. Thessaloniki (SKG): 465 PLN total - Depart KTW on Friday March 14, return Monday March 17

    Thessaloniki is the cheapest option, Athens is the capital and cultural wonder. Would you like more details on any of these options?
"""


In [ ]:
###############################################################################
# 9) Initialize openAI object
###############################################################################

openai = GPT_Wrapper()
openai.set_system_prompt(system_message)


In [ ]:
###############################################################################
# 10) Main Chat Function
###############################################################################

def chat(message, history):
    """
    The main chat loop that handles the conversation with the user,
    passing 'tools' definitions to the LLM for function calling.
    """
    print(f"inside chat function. Message: {message}")
    print(f"inside chat function. History: {history}")
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    try:
        openai.message_set(messages)
        response = openai.get_result_full(tools = tools_chatbot)
        print(f"inside chat function. Response: {response}")

        # If the LLM requests a function call, handle it
        while response.choices[0].finish_reason == "tool_calls":
            msg = response.choices[0].message
            print(f"[INFO] Tool call requested: {msg.tool_calls[0]}")
            tool_response, tool_args = handle_tool_call(msg)
            print(f"[INFO] Tool response: {tool_response}")

            # Add both the LLM's request and our tool response to the conversation
            messages.append(msg)
            messages.append(tool_response)

            # Re-send updated conversation to get final or next step
            openai.message_set(messages)
            response = openai.get_result_full()

        # Return normal text response (finish_reason = "stop")
        return response.choices[0].message.content

    except Exception as e:
        print(f"[ERROR] {e}")
        return "I'm sorry, something went wrong while processing your request."

In [ ]:
###############################################################################
# 11) Launch Gradio
###############################################################################
gr.ChatInterface(fn=chat, type="messages").launch()